<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/VirtualShell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import shlex
import readline  # Для удобного редактирования ввода
from pathlib import Path
from datetime import datetime

class VirtualFS:
    def __init__(self):
        self.root = {"/": {"type": "dir", "content": {}, "created": datetime.now()}}
        self.current_dir = "/"
        self.load_fs()  # Загружаем файловую систему при старте

    def save_fs(self):
        """Сохраняет файловую систему в файл"""
        with open("vfs_save.txt", "w") as f:
            f.write(self.serialize_fs())

    def load_fs(self):
        """Загружает файловую систему из файла"""
        if os.path.exists("vfs_save.txt"):
            with open("vfs_save.txt", "r") as f:
                data = f.read()
                if data:
                    self.deserialize_fs(data)

    def serialize_fs(self):
        """Сериализует файловую систему в строку"""
        import json
        return json.dumps({
            "root": self.root,
            "current_dir": self.current_dir
        }, default=str)  # str для сериализации datetime

    def deserialize_fs(self, data):
        """Десериализует файловую систему из строки"""
        import json
        loaded = json.loads(data)
        self.root = loaded["root"]
        self.current_dir = loaded["current_dir"]

        # Конвертируем строки времени обратно в datetime
        def fix_dates(obj):
            if isinstance(obj, dict):
                if "created" in obj:
                    obj["created"] = datetime.strptime(obj["created"], "%Y-%m-%d %H:%M:%S.%f")
                for v in obj.values():
                    fix_dates(v)
            elif isinstance(obj, list):
                for item in obj:
                    fix_dates(item)

        fix_dates(self.root)

    def get_absolute_path(self, path):
        if path.startswith("/"):
            return path
        return os.path.normpath(os.path.join(self.current_dir, path))

    def resolve_path(self, path):
        abs_path = self.get_absolute_path(path)
        parts = [p for p in abs_path.split("/") if p]

        current = self.root["/"]
        for part in parts:
            if part not in current["content"]:
                return None
            current = current["content"][part]
        return current

    def ls(self, path="."):
        target = self.resolve_path(path)
        if not target:
            return f"ls: cannot access '{path}': No such file or directory"
        if target["type"] != "dir":
            return f"ls: cannot access '{path}': Not a directory"

        items = []
        for name, item in target["content"].items():
            item_type = "dir" if item["type"] == "dir" else "file"
            created_time = item["created"].strftime("%Y-%m-%d %H:%M:%S")
            items.append(f"{name} [{item_type}] [{created_time}]")

        return "\n".join(items)

    def cat(self, path):
        target = self.resolve_path(path)
        if not target:
            return f"cat: {path}: No such file or directory"
        if target["type"] != "file":
            return f"cat: {path}: Is a directory"
        return target["content"]

    def edit(self, path):
        target = self.resolve_path(path)
        if not target:
            return f"edit: {path}: No such file or directory"
        if target["type"] != "file":
            return f"edit: {path}: Is a directory"

        print(f"Editing file: {path}")
        print("Current content:")
        print(target["content"])
        print("\nEnter new content (press Enter on empty line to save):")

        lines = []
        while True:
            try:
                line = input()
                if line == "":
                    break
                lines.append(line)
            except EOFError:
                break

        new_content = "\n".join(lines)
        target["content"] = new_content
        target["created"] = datetime.now()
        return "File saved successfully."

    def cd(self, path):
        if not path:
            path = "/"
        target = self.resolve_path(path)
        if not target:
            return f"cd: no such file or directory: {path}"
        if target["type"] != "dir":
            return f"cd: not a directory: {path}"
        self.current_dir = self.get_absolute_path(path)
        return ""

    def touch(self, path, content=""):
        abs_path = self.get_absolute_path(path)
        dirname, filename = os.path.split(abs_path)

        parent = self.resolve_path(dirname)
        if not parent or parent["type"] != "dir":
            return f"touch: cannot create file '{path}': No such directory"

        if filename in parent["content"]:
            parent["content"][filename]["created"] = datetime.now()
            return ""

        parent["content"][filename] = {
            "type": "file",
            "content": content,
            "created": datetime.now()
        }
        return ""

    def mkdir(self, path):
        abs_path = self.get_absolute_path(path)
        parts = [p for p in abs_path.split("/") if p]

        current = self.root["/"]
        for i, part in enumerate(parts):
            if part in current["content"]:
                if current["content"][part]["type"] != "dir":
                    return f"mkdir: cannot create directory '{path}': File exists"
                current = current["content"][part]
            else:
                for remaining_part in parts[i:]:
                    current["content"][remaining_part] = {
                        "type": "dir",
                        "content": {},
                        "created": datetime.now()
                    }
                    current = current["content"][remaining_part]
                break
        return ""

    def rm(self, path):
        abs_path = self.get_absolute_path(path)
        dirname, filename = os.path.split(abs_path)

        parent = self.resolve_path(dirname)
        if not parent or parent["type"] != "dir":
            return f"rm: cannot remove '{path}': No such directory"

        if filename not in parent["content"]:
            return f"rm: cannot remove '{path}': No such file or directory"

        if parent["content"][filename]["type"] == "dir" and parent["content"][filename]["content"]:
            return f"rm: cannot remove '{path}': Directory not empty"

        del parent["content"][filename]
        return ""

    def tree(self, path="/", indent=0, last=True, prefix=''):
        target = self.resolve_path(path)
        if not target:
            return f"tree: cannot access '{path}': No such file or directory"
        if target["type"] != "dir":
            return f"tree: '{path}' is not a directory"

        lines = []
        if indent == 0:
            lines.append(f"{path} [dir]")

        items = sorted(target["content"].items(), key=lambda x: x[0])
        count = len(items)

        for i, (name, item) in enumerate(items):
            is_last = i == count - 1
            new_prefix = prefix + ('    ' if last else '│   ')

            if item["type"] == "dir":
                lines.append(f"{prefix}{'└── ' if is_last else '├── '}{name} [dir]")
                lines.extend(self.tree(os.path.join(path, name), indent + 1, is_last, new_prefix).split('\n'))
            else:
                lines.append(f"{prefix}{'└── ' if is_last else '├── '}{name} [file]")

        return '\n'.join(filter(None, lines))

def print_help():
    print("Available commands:")
    print("  ls [path]          - List directory contents")
    print("  cat <file>         - Display file contents")
    print("  cd [path]          - Change directory")
    print("  touch <file> [content] - Create empty file or with content")
    print("  mkdir <dir>        - Create directory")
    print("  rm <file>          - Remove file")
    print("  edit <file>        - Edit file content interactively")
    print("  tree [path]        - Display directory tree structure")
    print("  exit               - Exit the shell (saves filesystem)")
    print("  help               - Show this help message")

def run_shell():
    vfs = VirtualFS()
    print("Virtual Shell. Type 'help' for available commands, 'exit' to quit.")

    while True:
        try:
            prompt = f"{vfs.current_dir} $ "
            cmd_input = input(prompt).strip()

            if not cmd_input:
                continue

            if cmd_input.lower() == "exit":
                vfs.save_fs()  # Сохраняем перед выходом
                break
            elif cmd_input.lower() == "help":
                print_help()
                continue

            parts = shlex.split(cmd_input)
            cmd = parts[0]
            args = parts[1:]

            if cmd == "ls":
                path = args[0] if args else "."
                result = vfs.ls(path)
                if result:
                    print(result)
            elif cmd == "cat":
                if not args:
                    print("cat: missing file operand")
                else:
                    result = vfs.cat(args[0])
                    print(result)
            elif cmd == "cd":
                path = args[0] if args else "/"
                result = vfs.cd(path)
                if result:
                    print(result)
            elif cmd == "touch":
                if not args:
                    print("touch: missing file operand")
                else:
                    content = " ".join(args[1:]) if len(args) > 1 else ""
                    result = vfs.touch(args[0], content)
                    if result:
                        print(result)
            elif cmd == "mkdir":
                if not args:
                    print("mkdir: missing directory operand")
                else:
                    result = vfs.mkdir(args[0])
                    if result:
                        print(result)
            elif cmd == "rm":
                if not args:
                    print("rm: missing operand")
                else:
                    result = vfs.rm(args[0])
                    if result:
                        print(result)
            elif cmd == "edit":
                if not args:
                    print("edit: missing file operand")
                else:
                    result = vfs.edit(args[0])
                    print(result)
            elif cmd == "tree":
                path = args[0] if args else "/"
                result = vfs.tree(path)
                print(result)
            else:
                print(f"{cmd}: command not found")

        except KeyboardInterrupt:
            print("\nUse 'exit' to quit")
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    run_shell()

Virtual Shell. Type 'help' for available commands, 'exit' to quit.
/ $ mkdir myfolder
/ $ ls
myfolder [dir] [2025-05-01 21:04:23]
/ $ touch new.txt
/ $ touch new1.txt hello
/ $ ls
myfolder [dir] [2025-05-01 21:04:23]
new.txt [file] [2025-05-01 21:04:48]
new1.txt [file] [2025-05-01 21:05:06]
/ $ tree
/ [dir]
├── myfolder [dir]
├── new.txt [file]
└── new1.txt [file]
/ $ cat new1.txt
hello
/ $ edit new.txt
Editing file: new.txt
Current content:


Enter new content (press Enter on empty line to save):
new content

File saved successfully.
/ $ ls
myfolder [dir] [2025-05-01 21:04:23]
new.txt [file] [2025-05-01 21:06:09]
new1.txt [file] [2025-05-01 21:05:06]
/ $ cat new.txt
new content
/ $ cd myfolder
/myfolder $ touch file
/myfolder $ cd ..
/ $ cd /
/ $ tree
/ [dir]
├── myfolder [dir]
    └── file [file]
├── new.txt [file]
└── new1.txt [file]
/ $ exit
